This is my "sandbox" to play around with the BEA api. My plan is to use this for my regional cycle project. Below I explain how to use the BEA api (stuff has changed since I last messed around with it)

In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
BEA_ID = "6BF79D8C-8042-4196-88DC-0E0C55B0C3B6"

After getting your key. [Here is some basic documentation](https://www.bea.gov/API/bea_web_service_api_user_guide.htm). Like a lot APIs, basic idea is to specify the URL in the proper way and it will return some output in a specified format. We can then convert the format (its .json) into a dataframe:



In [147]:
API_URL = "https://bea.gov/api/data?&UserID=6BF79D8C-8042-4196-88DC-0E0C55B0C3B6&method=GETDATASETLIST&ResultFormat=JSON&"
    
r = requests.get(API_URL)

In [152]:
type(r)


requests.models.Response

In [153]:
type(r.json())

dict

In [160]:
print(r.json().keys())

print(r.json()['BEAAPI'].keys())

print(r.json()['BEAAPI']["Results"].keys())

dict_keys(['BEAAPI'])
dict_keys(['Results', 'Request'])
dict_keys(['Dataset'])


This then leads to the insight that the `.json()` is a bunch of dictionaries within the dictionaries and that by working through it we can find a "root dictionary" that can be converted into a usable DataFrame

In [161]:
df = pd.DataFrame(r.json()["BEAAPI"]["Results"]['Dataset'])

In [162]:
df

,DatasetDescription,DatasetName
0,The RegionalData dataset is obsolete. Please u...,RegionalData
1,Standard NIPA tables,NIPA
2,Standard NI underlying detail tables,NIUnderlyingDetail
3,Multinational Enterprises,MNE
4,Standard Fixed Assets tables,FixedAssets
5,International Transactions Accounts,ITA
6,International Investment Position,IIP
7,GDP by Industry,GDPbyIndustry
8,Regional Income data sets,RegionalIncome
9,Regional Product data sets,RegionalProduct


Then this gives us the different datasets that are available through the BEA api. Now below, I'm going to grab personal income at the county level. The documentation for grabing this is [here](https://www.bea.gov/API/bea_web_service_api_user_guide.htm)

In [42]:
years = range(1969,2016)
years = list(years)

years = "".join(str(years))

years = years[1:-1]

In [43]:
years

'1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015'

In [44]:
my_key = "https://bea.gov/api/data?&UserID=6BF79D8C-8042-4196-88DC-0E0C55B0C3B6&method=GetData&"

data_set = "datasetname=RegionalIncome&"

table_and_line_income = "TableName=CA1&LineCode=3&"

table_and_line_population = "TableName=CA1&LineCode=2&"

year = "Year=" + years + "&"

location = "GeoFips=COUNTY&"

form = "ResultFormat=json"

In [45]:
API_URL = my_key + data_set + table_and_line_income + year + location + form

r = requests.get(API_URL)

df_income = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

In [46]:
df_income.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

df_income.rename(columns={"DataValue":"IncomePC"}, inplace=True)

df_income.head()

,IncomePC,GeoFips,GeoName,TimePeriod
0,28627,00000,United States,1999
1,31540,00000,United States,2001
2,38144,00000,United States,2006
3,19985,00000,United States,1991
4,21698,00000,United States,1993


In [47]:
API_URL = my_key + data_set + table_and_line_population + year + location + form

r = requests.get(API_URL)

population = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

population.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT", "GeoName"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

population.rename(columns={"DataValue":"Population"}, inplace=True)

population.head()

,Population,GeoFips,TimePeriod
0,306771529,00000,2009
1,219760875,00000,1977
2,215456585,00000,1975
3,266278393,00000,1995
4,233792014,00000,1983


In [48]:
combo = pd.merge(population, df_income,   # left df, right df
                 how='inner',      # Try the different options, inner, outer, left, right...what happens.
                 on=['GeoFips',"TimePeriod"],       # link with cntry
                 indicator=True)  # Tells us what happend

In [49]:
combo["TimePeriod"] = pd.to_datetime(combo["TimePeriod"], infer_datetime_format = True)

#combo.sort_values(by = "TimePeriod", inplace = True)

In [50]:
combo.set_index(["GeoFips","TimePeriod"],inplace = True)

In [51]:
combo.sort_index(level="GeoFips", inplace = True)

In [53]:
combo.head(50)

Population IncomePC        GeoName _merge
GeoFips TimePeriod                                          
00000   1969-01-01  201298000     3930  United States   both
        1970-01-01  203798722     4196  United States   both
        1971-01-01  206817509     4468  United States   both
        1972-01-01  209274882     4853  United States   both
        1973-01-01  211349205     5352  United States   both
        1974-01-01  213333635     5824  United States   both
        1975-01-01  215456585     6312  United States   both
        1976-01-01  217553859     6856  United States   both
        1977-01-01  219760875     7494  United States   both
        1978-01-01  222098244     8338  United States   both
        1979-01-01  224568579     9212  United States   both
        1980-01-01  227224719    10153  United States   both
        1981-01-01  229465744    11262  United States   both
        1982-01-01  231664432    11947  United States   both
        1983-01-01  233792014    12652  United States   both
        1984-01-01  235824907    13860  United States   both
        1985-01-01  237923734    14719  United States   both
        1986-01-01  240132831    15459  United States   both
        1987-01-01  242288936    16265  United States   both
        1988-01-01  244499004    17426  United States   both
        1989-01-01  246819222    18653  United States   both
        1990-01-01  249622814    19591  United States   both
        1991-01-01  252980941    19985  United States   both
        1992-01-01  256514224    21060  United States   both
        1993-01-01  259918588    21698  United States   both
        1994-01-01  263125821    22538  United States   both
        1995-01-01  266278393    23568  United States   both
        1996-01-01  269394284    24728  United States   both
        1997-01-01  272646925    25950  United States   both
        1998-01-01  275854104    27510  United States   both
        1999-01-01  279040168    28627  United States   both
        2000-01-01  282162411    30602  United States   both
        2001-01-01  284968955    31540  United States   both
        2002-01-01  287625193    31815  United States   both
        2003-01-01  290107933    32692  United States   both
        2004-01-01  292805298    34316  United States   both
        2005-01-01  295516599    35904  United States   both
        2006-01-01  298379912    38144  United States   both
        2007-01-01  301231207    39821  United States   both
        2008-01-01  304093966    41082  United States   both
        2009-01-01  306771529    39376  United States   both
        2010-01-01  309348193    40277  United States   both
        2011-01-01  311663358    42461  United States   both
        2012-01-01  313998379    44282  United States   both
        2013-01-01  316204908    44493  United States   both
        2014-01-01  318563456    46494  United States   both
        2015-01-01  320896618    48451  United States   both
01000   1969-01-01    3440000     2827        Alabama   both
        1970-01-01    3449846     3074        Alabama   both
        1971-01-01    3497452     3337        Alabama   both

In [57]:
combo.loc["02020"]

,Population,IncomePC,GeoName,_merge
TimePeriod,,,,
1969-01-01,123265,6232,"Anchorage Municipality, AK",both
1970-01-01,127569,6823,"Anchorage Municipality, AK",both
1971-01-01,134619,7102,"Anchorage Municipality, AK",both
1972-01-01,143203,7348,"Anchorage Municipality, AK",both
1973-01-01,147314,7853,"Anchorage Municipality, AK",both
1974-01-01,152386,9320,"Anchorage Municipality, AK",both
1975-01-01,165035,11439,"Anchorage Municipality, AK",both
1976-01-01,174496,12853,"Anchorage Municipality, AK",both
1977-01-01,177003,14071,"Anchorage Municipality, AK",both


In [288]:
df.DataValue.replace("(NA)", np.nan, inplace = True)

In [289]:
df.DataValue = df.DataValue.astype(float)

In [290]:
df.dtypes

CL_UNIT        object
Code           object
DataValue     float64
GeoFips        object
GeoName        object
NoteRef        object
TimePeriod     object
UNIT_MULT      object
dtype: object

In [291]:
df["TimePeriod"] = pd.to_datetime(df["TimePeriod"], infer_datetime_format = True)

In [292]:
df.sort_values(by = "TimePeriod", inplace = True)


In [294]:
df.head()

,CL_UNIT,Code,DataValue,GeoFips,GeoName,NoteRef,TimePeriod,UNIT_MULT
0,dollars,CA1-3,3930.0,00000,United States,NaN,1969-01-01,0
3535,dollars,CA1-3,2800.0,31123,"Morrill, NE",NaN,1969-01-01,0
3536,dollars,CA1-3,3093.0,31125,"Nance, NE",NaN,1969-01-01,0
3539,dollars,CA1-3,3727.0,31127,"Nemaha, NE",NaN,1969-01-01,0
3540,dollars,CA1-3,3106.0,31129,"Nuckolls, NE",NaN,1969-01-01,0


In [346]:
grp = df.groupby("GeoFips")

In [310]:
def log_diff_income(df):
    df["growth"] = np.log(df.DataValue).diff() / 46
    
    return df

In [388]:
transform_dict = {"DataValue": ["first","last", lambda x: np.diff(np.log(x))/46]}

new_df = grp.agg(transform_dict)

In [394]:
new_df["DataValue"]["last"].corr(new_df["DataValue"]["<lambda>"])

0.414791642511866

In [367]:
test =  new_df.diff(axis = 1).toseries

AttributeError: 'DataFrame' object has no attribute 'toseries'

In [385]:
np.diff?

In [309]:
grp = df.groupby("GeoFips")
grp.get_group("44007").TimePeriod.diff()

Object `TimePeriod.diff` not found.


In [ ]:
grp.get_group("44007").TimePeriod.diff

In [ ]:
grp.get_group("44007").sort_value

In [ ]:
grp.get_group("44007").sort_values

In [277]:
new_df.sort_values?

In [332]:
new_df.growth.corr(new_df.loc["1969-01-01"].DataValue)

nan

In [325]:
new_df.set_index("TimePeriod", inplace = True)

In [376]:
df.diff?